In [1]:
"""
Simple script for extracting Socrata Open Data Access (SODA) datasets. Compatible with 3+, though one can easily make it 2.7 
compatible by changing the `from urllib.error import HTTPError` import to `from urllib2 import HTTPError`
Parameters
----------
endpoint : string
  SODA API endpoint of the dataset.
count : int, default 1000
  The number of records to return in each paged result.
offset : int, default 0
  Offset the results returned. For example, if offset = 1000, the script will extract all records after the 1,000th record.
return_df : bool, default True
  Convert the returned SODA API dataset into a pandas DataFrame from JSON.
"""

import requests
import pandas as pd
from urllib.error import HTTPError # from urllib2 import HTTPError


def get_soda_api_data(endpoint, count=1000, offset=0, return_df=True):
    params = {'$limit': count, '$offset': offset}
    
    results = []

    while True:

        try:
            r = requests.get(endpoint, params=params)
            rcontent = r.json()

            if rcontent == []:
                break

            results.append(rcontent)
            offset += count
            params['$offset'] = offset

        except HTTPError as err:

            if err.response.status_code == '404':
                break
            else:
                print(err.response.status_code)
    
    if return_df:
        results_df = pd.DataFrame()

        for i in results:
            results_df = results_df.append(pd.io.json.json_normalize(i))
        
        return results_df
    
    else:
        return results

In [2]:
results_df = get_soda_api_data('https://data.austintexas.gov/resource/hcup-htgu.json', count=1000, offset=0, return_df=True)

<ipython-input-1-80074d4ea745>:50: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  results_df = results_df.append(pd.io.json.json_normalize(i))


In [3]:
results_df

,animal_id,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,name
0,A842851,2021-09-23T08:38:00.000,2021-09-23T08:38:00.000,2020-09-22T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,1 year,Fox,Gray/Red,NaN
1,A842843,2021-09-22T18:58:00.000,2021-09-22T18:58:00.000,2019-09-22T00:00:00.000,Return to Owner,NaN,Dog,Spayed Female,2 years,Staffordshire,Black/White,Gorda
2,A842842,2021-09-22T18:57:00.000,2021-09-22T18:57:00.000,2019-09-22T00:00:00.000,Return to Owner,NaN,Dog,Intact Male,2 years,Pit Bull,Blue/White,Kimbo
3,A842437,2021-09-22T18:40:00.000,2021-09-22T18:40:00.000,2021-04-16T00:00:00.000,Rto-Adopt,NaN,Cat,Spayed Female,5 months,Siamese Mix,Tortie Point,Puffy
4,A842234,2021-09-22T18:03:00.000,2021-09-22T18:03:00.000,2019-09-12T00:00:00.000,Adoption,NaN,Dog,Neutered Male,2 years,Anatol Shepherd,White,Max
...,...,...,...,...,...,...,...,...,...,...,...,...
543,A664223,2013-10-01T11:03:00.000,2013-10-01T11:03:00.000,2009-09-30T00:00:00.000,Return to Owner,NaN,Dog,Neutered Male,4 years,Bulldog Mix,White,Moby
544,A664237,2013-10-01T10:44:00.000,2013-10-01T10:44:00.000,2013-09-24T00:00:00.000,Transfer,Partner,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,NaN
545,A664236,2013-10-01T10:44:00.000,2013-10-01T10:44:00.000,2013-09-24T00:00:00.000,Transfer,Partner,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,NaN
546,A664235,2013-10-01T10:39:00.000,2013-10-01T10:39:00.000,2013-09-24T00:00:00.000,Transfer,Partner,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,NaN


In [4]:
results_df.to_csv('atxshelter2021.csv')

In [8]:
results_df.loc[results_df.breed=='Bat']

,animal_id,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,name
25,A842715,2021-09-22T11:50:00.000,2021-09-22T11:50:00.000,2020-09-20T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,1 year,Bat,Brown,NaN
363,A842388,2021-09-15T09:30:00.000,2021-09-15T09:30:00.000,2019-09-15T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,2 years,Bat,Brown,NaN
567,A842107,2021-09-10T10:42:00.000,2021-09-10T10:42:00.000,2019-09-09T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,2 years,Bat,Brown,NaN
577,A842110,2021-09-09T16:47:00.000,2021-09-09T16:47:00.000,2019-09-09T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,2 years,Bat,Brown,NaN
608,A842070,2021-09-09T10:31:00.000,2021-09-09T10:31:00.000,2019-09-08T00:00:00.000,Disposal,NaN,Other,Unknown,2 years,Bat,Brown,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
745,A665188,2013-10-15T08:16:00.000,2013-10-15T08:16:00.000,2013-09-14T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,4 weeks,Bat,Brown/Black,NaN
823,A665107,2013-10-13T19:00:00.000,2013-10-13T19:00:00.000,2011-10-13T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,2 years,Bat,Brown,NaN
824,A665106,2013-10-13T18:59:00.000,2013-10-13T18:59:00.000,2011-10-13T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,2 years,Bat,Brown,NaN
825,A665108,2013-10-13T18:58:00.000,2013-10-13T18:58:00.000,2011-10-13T00:00:00.000,Euthanasia,Rabies Risk,Other,Unknown,2 years,Bat,Brown,NaN


In [17]:
results_df.loc[results_df.breed=='Cockatoo']

,animal_id,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,name
876,A740126,2016-12-16T00:00:00.000,2016-12-16T00:00:00.000,1996-12-11T00:00:00.000,Transfer,Partner,Bird,Intact Male,20 years,Cockatoo,White/Orange,Sunny
878,A740127,2016-12-16T00:00:00.000,2016-12-16T00:00:00.000,1991-12-11T00:00:00.000,Transfer,Partner,Bird,Intact Female,25 years,Cockatoo,White/Orange,Sweet Pea
791,A733024,2016-08-18T09:21:00.000,2016-08-18T09:21:00.000,2012-08-16T00:00:00.000,Adoption,NaN,Bird,Intact Female,4 years,Cockatoo,White/Yellow,Cabana


jeff.ober@sparrow-partners.com